In [1]:
import sys
print(sys.executable)

import torch
print(torch.__version__)

import datasets
print(datasets.__version__)


c:\dev\toxic-ml\.venv\Scripts\python.exe


OSError: [WinError 1114] Une routine d’initialisation d’une bibliothèque de liens dynamiques (DLL) a échoué. Error loading "c:\dev\toxic-ml\.venv\Lib\site-packages\torch\lib\c10.dll" or one of its dependencies.

In [1]:
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer

from pathlib import Path
import sys

sys.path.append("../src/")

from preprocessing import preprocess

import nltk
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')

d:\envs\advanced_env_py311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\advanced_ml\Toxic-Comment-Classification-using-Classical-NLP-Methods-and-a-Lightweight-Transformer\notebooks\../src\preprocessing.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkgr
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package 

True

# Importing train set

7.1) page (12) 127,656 training samples, 15,957 validation samples, and 31,914 test samples

In [ ]:
# less than 1 minute to run
dataset = load_dataset("thesofakillers/jigsaw-toxic-comment-classification-challenge", split="train")

# Shuffle + use 10% of the data
dataset_10 = dataset.shuffle(seed=42).select(
    range(len(dataset) // 1000)          # 0,1% of the data
)

d:\envs\advanced_env_py311\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\hf_cache\hub\datasets--thesofakillers--jigsaw-toxic-comment-classification-challenge. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating test split: 100%|██████████| 306328/306328 [00:03<00:00, 100486.06 examples/s]


# Preprocessing

Conversion of emoticons into words, conversion of emojis into words, expansion of abbreviations and slang, conversion of non-ASCII characters, expansion of contractions, normalization of numeric expressions, case normalization, removal of HTML tags, URL processing, reduction of duplicated characters, sentence segmentation, lemmatization with POS tagging, lowercasing, punctuation removal, spelling correction, stopword removal, sentence concatenation, whitespace normalization, trimming of leading and trailing spaces

In [5]:
label_cols = [
    "toxic",
    "severe_toxic",
    "obscene",
    "threat",
    "insult",
    "identity_attack"
]

def process_row(example):
    example["comment_text"] = preprocess(example["comment_text"])
    # for col in label_cols:
    #     example[col] = int(example[col] > 0.5)
    return example


dataset = dataset_10.map(
    process_row,
    batched=False
)

Map: 100%|██████████| 159/159 [00:15<00:00, 10.51 examples/s]


In [6]:
df = dataset.to_pandas()

In [ ]:
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,1dd68a98d6b0f057,censorship science apologist know wiki rule po...,0,0,0,0,0,0
1,9d287bca4cff89ae,really cool someone get photo shakti huge cran...,0,0,0,0,0,0
2,ae8551a7bdbf0232,crazy look even respond personal attack guy co...,0,0,0,0,0,0
3,a9f2997e8dcc6cd1,thirty seven seven december two thousand ten e...,0,0,0,0,0,0
4,62e736877c28bf05,fuck masturbate,1,1,1,0,1,0
...,...,...,...,...,...,...,...,...
154,9ab5595cff28ba8e,bias moderator follow user moderate bias again...,1,0,0,0,0,0
155,d0b751c67d752d03,signature location courtesy note edit move sig...,0,0,0,0,0,0
156,5f2b194a5170271f,film image hold already release films image ap...,0,0,0,0,0,0
157,932e37af3869a6d9,thank strike rich prove immaturity,0,0,0,0,0,0


In [9]:
df[df["id"]=="9ab5595cff28ba8e"]["comment_text"].values[0]

'bias moderator follow user moderate bias against valid edits make page internode systems instead comment provide guidance contribution choose delete contribution denial normal reflect view lock page significant user unable contribute suck'

In [11]:
df_raw = dataset_10.to_pandas()
df_raw[df_raw["id"]=="9ab5595cff28ba8e"]["comment_text"].values[0]

"Biased Moderators \n\nThe following users overmoderate, , , . They are biased against valid edits made on pages such as Internode_Systems. Instead of commenting on and providing guidance on contributions, they choose to delete contributions that don't reflect their views. They then lock down the pages so users are unable to contribute to them. ,  and  can suck me off."